<center>
  <h2>Trabajo Laboratorio de Implementación III - MCD Virtual Cohorte 2022</h2>
  <h3>Forecasting para una Empresa de Consumo Masivo</h3>
  <h4>Creación de dataset group</h4>
</center>

In [1]:
# Importación librerias

# !pip install awswrangler
# !pip install boto3
import boto3
import awswrangler as wr
import pandas as pd
import numpy as np
import warnings
from datetime import datetime
from dateutil.relativedelta import relativedelta
warnings.filterwarnings("ignore")

In [2]:
# Definición fechas

date_start=datetime.strptime('2017-01-01', '%Y-%m-%d')
date_end=datetime.strptime('2019-12-01', '%Y-%m-%d')
forecast_length = 2
date_future=date_end+relativedelta(months=forecast_length)
years=range(2018, 2020)

print(f"Fecha inicio historia {date_start}")
print(f"Fecha fin historia {date_end}")
print(f"Fecha futura (related time series) {date_future}")

timestamp_format="yyyy-MM-dd"

# Versión modelo
data_version_cluster1 = '4_c1'
data_version_cluster2 = '4_c2'
data_version_cluster3 = '4_c3'
grouped_cols = "product_id"
grouped_cols_name = "product"

Fecha inicio historia 2017-01-01 00:00:00
Fecha fin historia 2019-12-01 00:00:00
Fecha futura (related time series) 2020-02-01 00:00:00


In [3]:
# Conexiones AWS

account_id=boto3.client('sts').get_caller_identity().get('Account')
session = boto3.Session() 
forecast = session.client(service_name='forecast')
region = forecast.meta.region_name
forecastquery = session.client(service_name='forecastquery')
role = f'arn:aws:iam::{account_id}:role/ForecastRole'

In [4]:
# Buckets de datos

# Buckets de datos

# CLUSTER 1
target_s3_path_c1=f's3://datasets-forecast-account1/modelo-{data_version_cluster1}-forecast-{grouped_cols_name}/target/df_ventas_{grouped_cols_name}.csv'
items_s3_path_c1=f's3://datasets-forecast-account1/modelo-{data_version_cluster1}-forecast-{grouped_cols_name}/product/df_products.csv'
related_s3_path_c1=f's3://datasets-forecast-account1/modelo-{data_version_cluster1}-forecast-{grouped_cols_name}/related/df_related_{grouped_cols_name}.csv'

# CLUSTER 2
target_s3_path_c2=f's3://datasets-forecast-account1/modelo-{data_version_cluster2}-forecast-{grouped_cols_name}/target/df_ventas_{grouped_cols_name}.csv'
items_s3_path_c2=f's3://datasets-forecast-account1/modelo-{data_version_cluster2}-forecast-{grouped_cols_name}/product/df_products.csv'
related_s3_path_c2=f's3://datasets-forecast-account1/modelo-{data_version_cluster2}-forecast-{grouped_cols_name}/related/df_related_{grouped_cols_name}.csv'

# CLUSTER 3
target_s3_path_c3=f's3://datasets-forecast-account1/modelo-{data_version_cluster3}-forecast-{grouped_cols_name}/target/df_ventas_{grouped_cols_name}.csv'
items_s3_path_c3=f's3://datasets-forecast-account1/modelo-{data_version_cluster3}-forecast-{grouped_cols_name}/product/df_products.csv'
related_s3_path_c3=f's3://datasets-forecast-account1/modelo-{data_version_cluster3}-forecast-{grouped_cols_name}/related/df_related_{grouped_cols_name}.csv'

In [5]:
# Verificar comunicación con Amazon Forecast
assert forecast.list_predictors()

In [6]:
# CLUSTER 1
# Nombres de datasets
ts_dataset_name_cluster1 = f"modelo_{data_version_cluster1}_{grouped_cols_name}_dataset_target"
item_dataset_name_cluster1 = f"modelo_{data_version_cluster1}_{grouped_cols_name}_dataset_items"
related_dataset_name_cluster1 = f"modelo_{data_version_cluster1}_{grouped_cols_name}_dataset_related"
# Nombre de dataset group
dataset_group_name_cluster1 = f'modelo_{data_version_cluster1}_{grouped_cols_name}'
# Nombre de import jobs
target_import_job_name_cluster1 = f"modelo_{data_version_cluster1}_{grouped_cols_name}_dataset_target_import"
items_import_job_name_cluster1 = f"modelo_{data_version_cluster1}_{grouped_cols_name}_dataset_items_import"
related_import_job_name_cluster1 = f"modelo_{data_version_cluster1}_{grouped_cols_name}_dataset_related_import"

# CLUSTER 2
# Nombres de datasets
ts_dataset_name_cluster2 = f"modelo_{data_version_cluster2}_{grouped_cols_name}_dataset_target"
item_dataset_name_cluster2 = f"modelo_{data_version_cluster2}_{grouped_cols_name}_dataset_items"
related_dataset_name_cluster2 = f"modelo_{data_version_cluster2}_{grouped_cols_name}_dataset_related"
# Nombre de dataset group
dataset_group_name_cluster2 = f'modelo_{data_version_cluster2}_{grouped_cols_name}'
# Nombre de import jobs
target_import_job_name_cluster2 = f"modelo_{data_version_cluster2}_{grouped_cols_name}_dataset_target_import"
items_import_job_name_cluster2 = f"modelo_{data_version_cluster2}_{grouped_cols_name}_dataset_items_import"
related_import_job_name_cluster2 = f"modelo_{data_version_cluster2}_{grouped_cols_name}_dataset_related_import"

# CLUSTER 3
# Nombres de datasets
ts_dataset_name_cluster3 = f"modelo_{data_version_cluster3}_{grouped_cols_name}_dataset_target"
item_dataset_name_cluster3 = f"modelo_{data_version_cluster3}_{grouped_cols_name}_dataset_items"
related_dataset_name_cluster3 = f"modelo_{data_version_cluster3}_{grouped_cols_name}_dataset_related"
# Nombre de dataset group
dataset_group_name_cluster3 = f'modelo_{data_version_cluster3}_{grouped_cols_name}'
# Nombre de import jobs
target_import_job_name_cluster3 = f"modelo_{data_version_cluster3}_{grouped_cols_name}_dataset_target_import"
items_import_job_name_cluster3 = f"modelo_{data_version_cluster3}_{grouped_cols_name}_dataset_items_import"
related_import_job_name_cluster3 = f"modelo_{data_version_cluster3}_{grouped_cols_name}_dataset_related_import"

<a id='datasets' />

## Crear datasets en Amazon Forecast
[(back to top)](#contents)

### Parámetros

Los datos deben ser importados sin header, por lo cual se debe definir un esquema. El orden de los atributos debe ser el mismo que el de las columnas de los archivos. 

### Target Time Series Schema

In [7]:
ts_frequency='M'
ts_schema = {
	"Attributes": [
		{
			"AttributeName": "timestamp",
			"AttributeType": "timestamp"
		},
		{
			"AttributeName": "item_id",
			"AttributeType": "string"
		},
		{
			"AttributeName": "target_value",
			"AttributeType": "float"
		}
	]
}

El siguiente codigo funciona para crear dataset target según schema brindado. La siguiente salida arroja error porque este primer modelo fue ejecutado integramente desde la UI de forecast

In [8]:
# CLUSTER 1
create_dataset_response_cluster1 = forecast.create_dataset(Domain="CUSTOM",
                                                  DatasetType='TARGET_TIME_SERIES',
                                                  DatasetName=ts_dataset_name_cluster1,
                                                  DataFrequency=ts_frequency,
                                                  Schema=ts_schema)

target_dataset_arn_cluster1 =create_dataset_response_cluster1['DatasetArn']
target_dataset_description_cluster1=forecast.describe_dataset(DatasetArn=target_dataset_arn_cluster1)

In [14]:
print(f"The Dataset with ARN {target_dataset_arn_cluster1} is now {target_dataset_description_cluster1['Status']}.")

The Dataset with ARN arn:aws:forecast:us-east-1:935182042847:dataset/modelo_4_c1_product_dataset_target is now ACTIVE.


In [10]:
# CLUSTER 2
create_dataset_response_cluster2 = forecast.create_dataset(Domain="CUSTOM",
                                                  DatasetType='TARGET_TIME_SERIES',
                                                  DatasetName=ts_dataset_name_cluster2,
                                                  DataFrequency=ts_frequency,
                                                  Schema=ts_schema)

target_dataset_arn_cluster2 =create_dataset_response_cluster2['DatasetArn']
target_dataset_description_cluster2=forecast.describe_dataset(DatasetArn=target_dataset_arn_cluster2)

In [13]:
print(f"The Dataset with ARN {target_dataset_arn_cluster2} is now {target_dataset_description_cluster2['Status']}.")

The Dataset with ARN arn:aws:forecast:us-east-1:935182042847:dataset/modelo_4_c2_product_dataset_target is now ACTIVE.


In [11]:
# CLUSTER 3
create_dataset_response_cluster3 = forecast.create_dataset(Domain="CUSTOM",
                                                  DatasetType='TARGET_TIME_SERIES',
                                                  DatasetName=ts_dataset_name_cluster3,
                                                  DataFrequency=ts_frequency,
                                                  Schema=ts_schema)

target_dataset_arn_cluster3 =create_dataset_response_cluster3['DatasetArn']
target_dataset_description_cluster3=forecast.describe_dataset(DatasetArn=target_dataset_arn_cluster3)

In [12]:
print(f"The Dataset with ARN {target_dataset_arn_cluster3} is now {target_dataset_description_cluster3['Status']}.")

The Dataset with ARN arn:aws:forecast:us-east-1:935182042847:dataset/modelo_4_c3_product_dataset_target is now ACTIVE.


### Items dataset schema

In [15]:
items_schema = {
	"Attributes": [
		{
			"AttributeName": "item_id",
			"AttributeType": "string"
		},
		{
			"AttributeName": "cat1",
			"AttributeType": "string"
		},
		{
			"AttributeName": "cat2",
			"AttributeType": "string"
		},
		{
			"AttributeName": "cat3",
			"AttributeType": "string"
		},
		{
			"AttributeName": "brand",
			"AttributeType": "string"
		},
		{
			"AttributeName": "sku_size",
			"AttributeType": "integer"
		}
	]
}

In [16]:
# CLUSTER 1
create_dataset_response_cluster1 = forecast.create_dataset(Domain="CUSTOM",
                                                  DatasetType='ITEM_METADATA',
                                                  DatasetName=item_dataset_name_cluster1,
                                                  Schema=items_schema)

items_dataset_arn_cluster1=create_dataset_response_cluster1['DatasetArn']
items_dataset_description_cluster1=forecast.describe_dataset(DatasetArn=items_dataset_arn_cluster1)
print(f"The Dataset with ARN {items_dataset_arn_cluster1} is now {items_dataset_description_cluster1['Status']}.")

The Dataset with ARN arn:aws:forecast:us-east-1:935182042847:dataset/modelo_4_c1_product_dataset_items is now ACTIVE.


In [17]:
# CLUSTER 2
create_dataset_response_cluster2 = forecast.create_dataset(Domain="CUSTOM",
                                                  DatasetType='ITEM_METADATA',
                                                  DatasetName=item_dataset_name_cluster2,
                                                  Schema=items_schema)

items_dataset_arn_cluster2=create_dataset_response_cluster2['DatasetArn']
items_dataset_description_cluster2=forecast.describe_dataset(DatasetArn=items_dataset_arn_cluster2)
print(f"The Dataset with ARN {items_dataset_arn_cluster2} is now {items_dataset_description_cluster2['Status']}.")

The Dataset with ARN arn:aws:forecast:us-east-1:935182042847:dataset/modelo_4_c2_product_dataset_items is now ACTIVE.


In [18]:
# CLUSTER 3
create_dataset_response_cluster3 = forecast.create_dataset(Domain="CUSTOM",
                                                  DatasetType='ITEM_METADATA',
                                                  DatasetName=item_dataset_name_cluster3,
                                                  Schema=items_schema)

items_dataset_arn_cluster3=create_dataset_response_cluster3['DatasetArn']
items_dataset_description_cluster3=forecast.describe_dataset(DatasetArn=items_dataset_arn_cluster3)
print(f"The Dataset with ARN {items_dataset_arn_cluster3} is now {items_dataset_description_cluster3['Status']}.")

The Dataset with ARN arn:aws:forecast:us-east-1:935182042847:dataset/modelo_4_c3_product_dataset_items is now ACTIVE.


### Related dataset schema

In [19]:
related_dataset_frequency='M'
related_schema = {
	"Attributes": [
		{
			"AttributeName": "timestamp",
			"AttributeType": "timestamp"
		},
		{
			"AttributeName": "item_id",
			"AttributeType": "string"
		},
		{
			"AttributeName": "month_number",
			"AttributeType": "integer"
		},
		{
			"AttributeName": "quarter",
			"AttributeType": "integer"
		},
		{
			"AttributeName": "days_in_month",
			"AttributeType": "integer"
		},
		{
			"AttributeName": "sundays",
			"AttributeType": "integer"
		},
		{
			"AttributeName": "saturdays",
			"AttributeType": "integer"
		},
		{
			"AttributeName": "tn_m3",
			"AttributeType": "float"
		},
		{
			"AttributeName": "tn_m6",
			"AttributeType": "float"
		},
		{
			"AttributeName": "tn_m12",
			"AttributeType": "float"
		}
	]
}

In [20]:
# CLUSTER 1
create_dataset_response_cluster1 = forecast.create_dataset(Domain="CUSTOM",
                                                  DatasetType='RELATED_TIME_SERIES',
                                                  DatasetName=related_dataset_name_cluster1,
                                                  DataFrequency=related_dataset_frequency,
                                                  Schema=related_schema)
related_dataset_arn_cluster1=create_dataset_response_cluster1['DatasetArn']
related_dataset_description_cluster1=forecast.describe_dataset(DatasetArn=related_dataset_arn_cluster1)
print(f"The Dataset with ARN {related_dataset_arn_cluster1} is now {related_dataset_description_cluster1['Status']}.")

The Dataset with ARN arn:aws:forecast:us-east-1:935182042847:dataset/modelo_4_c1_product_dataset_related is now ACTIVE.


In [21]:
# CLUSTER 2
create_dataset_response_cluster2 = forecast.create_dataset(Domain="CUSTOM",
                                                  DatasetType='RELATED_TIME_SERIES',
                                                  DatasetName=related_dataset_name_cluster2,
                                                  DataFrequency=related_dataset_frequency,
                                                  Schema=related_schema)
related_dataset_arn_cluster2=create_dataset_response_cluster2['DatasetArn']
related_dataset_description_cluster2=forecast.describe_dataset(DatasetArn=related_dataset_arn_cluster2)
print(f"The Dataset with ARN {related_dataset_arn_cluster2} is now {related_dataset_description_cluster2['Status']}.")

The Dataset with ARN arn:aws:forecast:us-east-1:935182042847:dataset/modelo_4_c2_product_dataset_related is now ACTIVE.


In [22]:
# CLUSTER 3
create_dataset_response_cluster3 = forecast.create_dataset(Domain="CUSTOM",
                                                  DatasetType='RELATED_TIME_SERIES',
                                                  DatasetName=related_dataset_name_cluster3,
                                                  DataFrequency=related_dataset_frequency,
                                                  Schema=related_schema)
related_dataset_arn_cluster3=create_dataset_response_cluster3['DatasetArn']
related_dataset_description_cluster3=forecast.describe_dataset(DatasetArn=related_dataset_arn_cluster3)
print(f"The Dataset with ARN {related_dataset_arn_cluster3} is now {related_dataset_description_cluster3['Status']}.")

The Dataset with ARN arn:aws:forecast:us-east-1:935182042847:dataset/modelo_4_c3_product_dataset_related is now ACTIVE.


### Dataset Group

Para entrenar un predictor, requerimos un `DatasetGroup` que agrupa los datasets de input. 

In [23]:
# CLUSTER1
dataset_arns_cluster1 = [target_dataset_arn_cluster1, items_dataset_arn_cluster1, related_dataset_arn_cluster1]
create_dataset_group_response_cluster1 = \
    forecast.create_dataset_group(Domain="CUSTOM",
                                  DatasetGroupName=dataset_group_name_cluster1,
                                  DatasetArns=dataset_arns_cluster1)
dataset_group_arn_cluster1 = create_dataset_group_response_cluster1['DatasetGroupArn']
describe_dataset_group_response_cluster1 = forecast.describe_dataset_group(DatasetGroupArn=dataset_group_arn_cluster1)
print(f"The DatasetGroup with ARN {dataset_group_arn_cluster1} is now {describe_dataset_group_response_cluster1['Status']}.")

The DatasetGroup with ARN arn:aws:forecast:us-east-1:935182042847:dataset-group/modelo_4_c1_product is now ACTIVE.


In [24]:
# cluster2
dataset_arns_cluster2 = [target_dataset_arn_cluster2, items_dataset_arn_cluster2, related_dataset_arn_cluster2]
create_dataset_group_response_cluster2 = \
    forecast.create_dataset_group(Domain="CUSTOM",
                                  DatasetGroupName=dataset_group_name_cluster2,
                                  DatasetArns=dataset_arns_cluster2)
dataset_group_arn_cluster2 = create_dataset_group_response_cluster2['DatasetGroupArn']
describe_dataset_group_response_cluster2 = forecast.describe_dataset_group(DatasetGroupArn=dataset_group_arn_cluster2)
print(f"The DatasetGroup with ARN {dataset_group_arn_cluster2} is now {describe_dataset_group_response_cluster2['Status']}.")

The DatasetGroup with ARN arn:aws:forecast:us-east-1:935182042847:dataset-group/modelo_4_c2_product is now ACTIVE.


In [25]:
# cluster3
dataset_arns_cluster3 = [target_dataset_arn_cluster3, items_dataset_arn_cluster3, related_dataset_arn_cluster3]
create_dataset_group_response_cluster3 = \
    forecast.create_dataset_group(Domain="CUSTOM",
                                  DatasetGroupName=dataset_group_name_cluster3,
                                  DatasetArns=dataset_arns_cluster3)
dataset_group_arn_cluster3 = create_dataset_group_response_cluster3['DatasetGroupArn']
describe_dataset_group_response_cluster3 = forecast.describe_dataset_group(DatasetGroupArn=dataset_group_arn_cluster3)
print(f"The DatasetGroup with ARN {dataset_group_arn_cluster3} is now {describe_dataset_group_response_cluster3['Status']}.")

The DatasetGroup with ARN arn:aws:forecast:us-east-1:935182042847:dataset-group/modelo_4_c3_product is now ACTIVE.


<a id='import' />

## Importar datasets en Amazon Forecast
[(back to top)](#contents)

### Target time series import job

In [26]:
# cluster1
target_dataset_import_job_response_cluster1 = \
    forecast.create_dataset_import_job(DatasetImportJobName=target_import_job_name_cluster1,
                                       DatasetArn=target_dataset_arn_cluster1,
                                       DataSource= {
                                         "S3Config" : {
                                             "Path": target_s3_path_c1,
                                             "RoleArn": role
                                         } 
                                       },
                                       TimestampFormat=timestamp_format)

target_dataset_import_job_arn_cluster1 = target_dataset_import_job_response_cluster1['DatasetImportJobArn']
forecast.describe_dataset_import_job(DatasetImportJobArn=target_dataset_import_job_arn_cluster1)['Status']

'CREATE_PENDING'

In [44]:
forecast.describe_dataset_import_job(DatasetImportJobArn=target_dataset_import_job_arn_cluster1)['Status']

'ACTIVE'

In [27]:
# cluster2
target_dataset_import_job_response_cluster2 = \
    forecast.create_dataset_import_job(DatasetImportJobName=target_import_job_name_cluster2,
                                       DatasetArn=target_dataset_arn_cluster2,
                                       DataSource= {
                                         "S3Config" : {
                                             "Path": target_s3_path_c2,
                                             "RoleArn": role
                                         } 
                                       },
                                       TimestampFormat=timestamp_format)

target_dataset_import_job_arn_cluster2 = target_dataset_import_job_response_cluster2['DatasetImportJobArn']
forecast.describe_dataset_import_job(DatasetImportJobArn=target_dataset_import_job_arn_cluster2)['Status']

'CREATE_PENDING'

In [45]:
forecast.describe_dataset_import_job(DatasetImportJobArn=target_dataset_import_job_arn_cluster2)['Status']

'ACTIVE'

In [28]:
# cluster3
target_dataset_import_job_response_cluster3 = \
    forecast.create_dataset_import_job(DatasetImportJobName=target_import_job_name_cluster3,
                                       DatasetArn=target_dataset_arn_cluster3,
                                       DataSource= {
                                         "S3Config" : {
                                             "Path": target_s3_path_c3,
                                             "RoleArn": role
                                         } 
                                       },
                                       TimestampFormat=timestamp_format)

target_dataset_import_job_arn_cluster3 = target_dataset_import_job_response_cluster3['DatasetImportJobArn']
forecast.describe_dataset_import_job(DatasetImportJobArn=target_dataset_import_job_arn_cluster3)['Status']

'CREATE_PENDING'

In [46]:
forecast.describe_dataset_import_job(DatasetImportJobArn=target_dataset_import_job_arn_cluster3)['Status']

'ACTIVE'

### Items dataset import job

In [47]:
# cluster1
items_dataset_import_job_response_cluster1 = \
    forecast.create_dataset_import_job(DatasetImportJobName=items_import_job_name_cluster1,
                                       DatasetArn=items_dataset_arn_cluster1,
                                       DataSource= {
                                         "S3Config" : {
                                             "Path": items_s3_path_c1,
                                             "RoleArn": role
                                         } 
                                       })

items_dataset_import_job_arn_cluster1 = items_dataset_import_job_response_cluster1['DatasetImportJobArn']
forecast.describe_dataset_import_job(DatasetImportJobArn=items_dataset_import_job_arn_cluster1)['Status']

'CREATE_PENDING'

In [48]:
# cluster2
items_dataset_import_job_response_cluster2 = \
    forecast.create_dataset_import_job(DatasetImportJobName=items_import_job_name_cluster2,
                                       DatasetArn=items_dataset_arn_cluster2,
                                       DataSource= {
                                         "S3Config" : {
                                             "Path": items_s3_path_c2,
                                             "RoleArn": role
                                         } 
                                       })

items_dataset_import_job_arn_cluster2 = items_dataset_import_job_response_cluster2['DatasetImportJobArn']
forecast.describe_dataset_import_job(DatasetImportJobArn=items_dataset_import_job_arn_cluster2)['Status']

'CREATE_PENDING'

In [49]:
# cluster3
items_dataset_import_job_response_cluster3 = \
    forecast.create_dataset_import_job(DatasetImportJobName=items_import_job_name_cluster3,
                                       DatasetArn=items_dataset_arn_cluster3,
                                       DataSource= {
                                         "S3Config" : {
                                             "Path": items_s3_path_c3,
                                             "RoleArn": role
                                         } 
                                       })

items_dataset_import_job_arn_cluster3 = items_dataset_import_job_response_cluster3['DatasetImportJobArn']
forecast.describe_dataset_import_job(DatasetImportJobArn=items_dataset_import_job_arn_cluster3)['Status']

'CREATE_PENDING'

### Related dataset import job

In [51]:
# cluster1
related_dataset_import_job_response_cluster1 = \
    forecast.create_dataset_import_job(DatasetImportJobName=related_import_job_name_cluster1,
                                       DatasetArn=related_dataset_arn_cluster1,
                                       DataSource= {
                                         "S3Config" : {
                                             "Path": related_s3_path_c1,
                                             "RoleArn": role
                                         } 
                                       },
                                       TimestampFormat=timestamp_format)

related_dataset_import_job_arn_cluster1 = related_dataset_import_job_response_cluster1['DatasetImportJobArn']
forecast.describe_dataset_import_job(DatasetImportJobArn=related_dataset_import_job_arn_cluster1)['Status']

'CREATE_PENDING'

In [52]:
# cluster2
related_dataset_import_job_response_cluster2 = \
    forecast.create_dataset_import_job(DatasetImportJobName=related_import_job_name_cluster2,
                                       DatasetArn=related_dataset_arn_cluster2,
                                       DataSource= {
                                         "S3Config" : {
                                             "Path": related_s3_path_c2,
                                             "RoleArn": role
                                         } 
                                       },
                                       TimestampFormat=timestamp_format)

related_dataset_import_job_arn_cluster2 = related_dataset_import_job_response_cluster2['DatasetImportJobArn']
forecast.describe_dataset_import_job(DatasetImportJobArn=related_dataset_import_job_arn_cluster2)['Status']

'CREATE_PENDING'

In [53]:
# cluster3
related_dataset_import_job_response_cluster3 = \
    forecast.create_dataset_import_job(DatasetImportJobName=related_import_job_name_cluster3,
                                       DatasetArn=related_dataset_arn_cluster3,
                                       DataSource= {
                                         "S3Config" : {
                                             "Path": related_s3_path_c3,
                                             "RoleArn": role
                                         } 
                                       },
                                       TimestampFormat=timestamp_format)

related_dataset_import_job_arn_cluster3 = related_dataset_import_job_response_cluster3['DatasetImportJobArn']
forecast.describe_dataset_import_job(DatasetImportJobArn=related_dataset_import_job_arn_cluster3)['Status']

'CREATE_PENDING'

In [57]:
forecast.describe_dataset_import_job(DatasetImportJobArn=related_dataset_import_job_arn_cluster1)['Status']

'ACTIVE'